In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from utils import ROOT_DIR, get_data
from joblib import load

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = get_data()

In [3]:
data = {
    "training": (X_train, Y_train),
    "validation": (X_validation, Y_validation),
    "test": (X_test, Y_test)
}

In [4]:
sets = {
    "training": None,
    "validation": None,
    "test": None
}

In [5]:
description = {
    "model": None,
    "predictions": {**sets},
    "rmse": {**sets},
    "mae": {**sets},
    "R2": {**sets}
}

In [6]:
models = {
    "MLP0": {**description},
    "MLP1": {**description},
    "MLP2": {**description}
}

In [7]:
models["MLP0"]["model"] = tf.keras.models.load_model(ROOT_DIR + "/best-models/mlp/hidden-layers-0")
models["MLP1"]["model"] = tf.keras.models.load_model(ROOT_DIR + "/best-models/mlp/hidden-layers-1")
models["MLP2"]["model"] = tf.keras.models.load_model(ROOT_DIR + "/best-models/mlp/hidden-layers-2")

In [8]:
rmseDf = pd.DataFrame(index=[model_key for model_key, _ in models.items()])

In [9]:
maeDf = pd.DataFrame(index=[model_key for model_key, _ in models.items()])

In [10]:
R2Df = pd.DataFrame(index=[model_key for model_key, _ in models.items()])

In [11]:
for model_key, model_description in models.items():
    for set_key, set_value in model_description["predictions"].items():
        model_description["predictions"][set_key] = model_description["model"].predict(data[set_key][0]).reshape(-1)
        model_description["rmse"][set_key] = np.sqrt(mean_squared_error(data[set_key][1], model_description["predictions"][set_key]))
        model_description["mae"][set_key] = mean_absolute_error(data[set_key][1], model_description["predictions"][set_key])
        model_description["R2"][set_key] = r2_score(data[set_key][1], model_description["predictions"][set_key])
        
        rmseDf.loc[model_key, set_key] = model_description["rmse"][set_key]
        maeDf.loc[model_key, set_key] = model_description["mae"][set_key]
        R2Df.loc[model_key, set_key] = model_description["R2"][set_key]

In [12]:
print(rmseDf.to_latex(caption="RMSE metric of models for training/validation/test sets",
                      label="tab:rmse-metrics",
                      formatters=["{:0.2f}".format, "{:0.2f}".format, "{:0.2f}".format]))

\begin{table}
\centering
\caption{RMSE metric of models for training/validation/test sets}
\label{tab:rmse-metrics}
\begin{tabular}{lrrr}
\toprule
{} & training & validation & test \\
\midrule
MLP0 &     0.70 &       0.68 & 0.66 \\
MLP1 &     0.63 &       0.62 & 0.63 \\
MLP2 &     0.68 &       0.63 & 0.66 \\
\bottomrule
\end{tabular}
\end{table}



In [13]:
print(maeDf.to_latex(caption="MAE metric of models for training/validation/test sets",
                     label="tab:mae-metrics",
                     formatters=["{:0.2f}".format, "{:0.2f}".format, "{:0.2f}".format]))

\begin{table}
\centering
\caption{MAE metric of models for training/validation/test sets}
\label{tab:mae-metrics}
\begin{tabular}{lrrr}
\toprule
{} & training & validation & test \\
\midrule
MLP0 &     0.45 &       0.45 & 0.45 \\
MLP1 &     0.39 &       0.40 & 0.40 \\
MLP2 &     0.41 &       0.42 & 0.43 \\
\bottomrule
\end{tabular}
\end{table}



In [14]:
print(R2Df.to_latex(caption="R2 metric of models for training/validation/test sets",
                             label="tab:r2-metrics",
                             formatters=["{:0.3f}".format, "{:0.3f}".format, "{:0.3f}".format]))

\begin{table}
\centering
\caption{R2 metric of models for training/validation/test sets}
\label{tab:r2-metrics}
\begin{tabular}{lrrr}
\toprule
{} & training & validation &  test \\
\midrule
MLP0 &    0.227 &      0.111 & 0.242 \\
MLP1 &    0.361 &      0.279 & 0.321 \\
MLP2 &    0.271 &      0.239 & 0.255 \\
\bottomrule
\end{tabular}
\end{table}

